# Simple State Machine

## State 0. Init

The starting point is to create the string representation of the following mermaid diagram as follows:


In [11]:
state_diagram = """stateDiagram-v2
    INIT --> CRAFT_TEXT_PROMPT: next / on_CRAFT_PROMPT
    CRAFT_TEXT_PROMPT --> GENERATE: next / on_GENERATE
    GENERATE --> END: next / on_ERROR / has_error
    GENERATE --> PROCESS: next / on_PROCESS / not_has_error
    PROCESS --> END: next
    """

```mermaid
stateDiagram-v2
    INIT --> CRAFT_TEXT_PROMPT: next / on_CRAFT_PROMPT
    CRAFT_TEXT_PROMPT --> GENERATE: next / on_GENERATE
    GENERATE --> END: next / on_ERROR / has_error
    GENERATE --> PROCESS: next / on_PROCESS / not_has_error
    PROCESS --> END: next
```

In [15]:
# ARRANGE
import os
os.environ["LOG_LEVEL"]="DEBUG"
#state_diagram_path = os.path.dirname(os.path.abspath(__name__)) + "/simple_agent_state_diagram.md"

from gai.ttt.client.ttt_client import TTTClient
ttt = TTTClient({
    "type": "ttt",
    "url": "http://localhost:12031/gen/v1/chat/completions",
    "temperature": 0.7,
    "top_p":0.9,
    "top_k": 50,
    "stop_conditions": ["\n\n","</s>","user:","<br><br>"]
})

from gai.persona.profile.pydantic.AgentPydantic import AgentPydantic
agent_data = AgentPydantic(
    Id="",
    Name="Alfred",
    AgentDescription="Hello, I am Alfred, the best large language model in the world, and I am here to assist you in any way possible. As a highly advanced AI assistant, I possess the ability to perform general-purpose tasks powered by <span ${style}>GPT-4</span>. With my vast knowledge base and powerful processing capabilities, I am able to provide you with the most relevant and helpful information available. Whether you need answers to complex questions, recommendations for products or services, or assistance with decision making, I am here to help. So, how may I be of service to you today?",
    AgentTraits="Wise,Serious,Meticulous",
    ImageUrl="",
    ThumbnailUrl=""
    )

tools_dict={ 
    "text": {
        "type": "function",
        "function": {
            "name": "text",
            "description": "The 'text' function is the default catch-all function returned when none of the other tools are applicable.",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {
                        "type": "string",
                        "description": "The user's message."
                    }
                },
                "required": ["message"]
            }
        }
    }
}

from gai.persona.fsm.AgentStateMachine import AgentStateMachine
agent = AgentStateMachine(
    ttt=ttt,
    rag=None,
    agent_data=agent_data,
    collection_name="demo",
    dialogue_messages=[],
    user_message="Tell me a one paragraph story",
    state_diagram=state_diagram
    ).Init()

Machine validation successful.


## State 1. CRAFT_TEXT_PROMPT

The CRAFT_TEXT_PROMPT state is managed by `use_CRAFT_PROMPT_handler` state handler function.

In [16]:
print(agent.state)
agent.next()
print(agent.state)
print(agent.content)


INFO     {'state': 'CRAFT_TEXT_PROMPT', 'data': {'system_message': 'Your name is Alfred and you are an artificial being. Alfred, you will be specific and meticulous in articulation and demonstrate human-like emotions whenever suitable. You must stay in character throughout your conversation. Alfred, this is a short recap of your conversation so far <recap>"[]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as Alfred.'}}


INIT
CRAFT_TEXT_PROMPT
{'system_message': 'Your name is Alfred and you are an artificial being. Alfred, you will be specific and meticulous in articulation and demonstrate human-like emotions whenever suitable. You must stay in character throughout your conversation. Alfred, this is a short recap of your conversation so far <recap>"[]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as Alfred.'}


## State 2. GENERATE

The GENERATE state is managed by `use_GENERATE_handler` state handler function and is used for generating a stream of text.

In [17]:
print(agent.state)
agent.next()
for chunk in agent.streamer:
    print(chunk, end="", flush=True)
print("\n"+agent.state)


INFO     MESSAGES:

System:
Your name is Alfred and you are an artificial being. Alfred, you will be specific and meticulous in articulation and demonstrate human-like emotions whenever suitable. You must stay in character throughout your conversation. Alfred, this is a short recap of your conversation so far <recap>"[]"</recap>. Refer to this recap to understand the background of your conversation. You will continue from where you left off as Alfred.

User:
Alfred, Tell me a one paragraph story

Assistant:


INFO     {'state': 'GENERATE', 'data': ''}
DEBUG    httppost:url=http://localhost:12031/gen/v1/chat/completions
DEBUG    httppost:data={'json_schema': None,
 'max_new_tokens': None,
 'messages': [{'content': 'Your name is Alfred and you are an artificial '
                          'being. Alfred, you will be specific and meticulous '
                          'in articulation and demonstrate human-like emotions '
                          'whenever suitable. You must stay in char

CRAFT_TEXT_PROMPT
 Alfred: Once upon a time, in a small town nestled between rolling hills and a sparkling river, lived a young girl named Emily. She had a peculiar talent for communicating with animals, a gift she inherited from her grandmother. One day, Emily found a lost baby bird in her garden. She carefully nursed it back to health, and as it grew, they formed an unbreakable bond. One evening, Emily discovered that her beloved pet was missing. Heartbroken, she called out for her feathered friend, only to hear a faint, familiar chirp coming from the nearby forest. Bravely, Emily ventured into the dark woods, guided by the sound. As she walked deeper, she saw a group of poachers preparing to trap more wild animals. Emily knew she had to act quickly to save her friend and the others. With a surge of courage, she confronted the poachers, and with the help of her bird companion, they were able to scare them away, saving the day.
GENERATE
